In [ ]:
from ckonlpy.tag import Twitter, Postprocessor
import pandas as pd
import numpy as np

In [ ]:
'''
데이터 불러오기 및 결측치 확인
'''
fixed_data_df = pd.read_csv('./data/original_data.csv')
fixed_data_df = fixed_data_df
fixed_data_df.isnull().sum()
fixed_data_df.shape

'''
공백 제거
'''
fixed_data_df['document'] = fixed_data_df['document'].str.strip()

'''
정규 표현식 사용
'''
fixed_data_df['document'] = fixed_data_df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
fixed_data_df['document'].replace('', np.nan, inplace = True) # 결측치가 아닌 공백을 결측치로 치환

'''
결측치 제거
'''
# fixed_data_df.loc[fixed_data_Df.document.isnull()][:5] # 결측치 출력
fixed_data_df = fixed_data_df.dropna(axis = 0)
fixed_data_df.isnull().sum()
fixed_data_df.shape

'''
데이터 저장, 198,884개 문장
'''
fixed_data_df.to_csv('./data/fixed_data.csv')

'''
단어 토큰화, 명사 추출
'''
stopwords = {'의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다', '어요'}
passtags = {'Noun'}

twitter = Twitter()
postprocessor = Postprocessor(
    base_tagger = twitter, # base tagger
    stopwords = stopwords, # 해당 단어 필터
    #passwords = passwords, # 해당 단어만 선택
    passtags = passtags, # 해당 품사만 선택
    #replace = replace, # 해당 단어 set 치환
    #ngrams = ngrams # 해당 복합 단어 set을 한 단어로 결합
)
fixed_data_df = pd.read_csv('./data/fixed_data.csv')

In [ ]:
sentence_list = fixed_data_df['document'].to_list()

In [ ]:
tokenized_nouns_list = []
i = 0
for sentence in sentence_list:
    tokenized_word = postprocessor.pos(sentence)
    for word in tokenized_word:
        tokenized_nouns_list.append(word[0])
    i += 1
    print("{}/{}".format(i, len(sentence_list)))

In [ ]:
'''
중복단어 체크
'''
word_dict = {}
for i in tokenized_nouns_list:
    if i in word_dict:
        word_dict[i] += 1
    else: 
        word_dict[i] = 1
        
print(len(word_dict))

In [ ]:
'''
빈도수 1~3인 단어 삭제
'''
fixed_word_dict = {key: value for key, value in word_dict.items() if value != 1}
fixed_word_dict = {key: value for key, value in fixed_word_dict.items() if value != 2}
fixed_word_dict = {key: value for key, value in fixed_word_dict.items() if value != 3}
fixed_word_dict = {key: value for key, value in fixed_word_dict.items() if len(key) != 1}
print(len(fixed_word_dict))

In [ ]:
'''
데이터프레임 변환
'''
nouns_list = list(fixed_word_dict.keys())
counts_list = list(fixed_word_dict.values())
counted_nouns_df = pd.DataFrame({'nouns': nouns_list, 'counts_list': counts_list})

In [ ]:
'''
count 기준 내림차순 정렬 및 인덱스 초기화
'''
counted_nouns_df = counted_nouns_df.sort_values(by='counts_list', ascending=False).reset_index(drop = True)

In [ ]:
counted_nouns_df[:20]

In [ ]:
'''
저장
'''
counted_nouns_df.to_csv('./data/fixed_word_df.csv', index = False)